## Mapping the Data

In [2]:
import pandas as pd
import requests

In [ ]:
# pip install geopandas
# pip install geopy

In [6]:
import geopandas as gpd

In [7]:
pip install geopy

     |████████████████████████████████| 119 kB 6.4 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 29.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [13]:
import geopy
from geopy.geocoders import Nominatim

In [14]:
# Test geocoding
locator = Nominatim(user_agent= "myGeocoder")
location = locator.geocode("Champ de Mars, Paris, France")

In [16]:
print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))

Latitude = 48.85614465, Longitude = 2.297820393322227


In [ ]:
# Geocoding addresses from Pandas
